In [1]:
from __future__ import print_function
from hyperopt import Trials, STATUS_OK, tpe, rand
from hyperas import optim
from hyperas.distributions import choice, uniform

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, Activation
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam

from keras.utils import np_utils
from keras import backend as K

def data():
    
    import datetime as dt
    import numpy as np
    import pandas as pd
    from sklearn.preprocessing import MinMaxScaler
    
    df = pd.read_json('df.json', orient='split')
    df = pd.DataFrame(df.eth_close.copy())
    df = df[557:]
    
    scaler_1 = MinMaxScaler(feature_range=(0,1))
    df_scaled = scaler_1.fit_transform(df.values)
    df = pd.DataFrame(df_scaled).set_index(df.index.values)

    df.columns = ['unshifted']
    n_shifts=1
    df_shifted = df.copy()
    for i in range(n_shifts):
        df = pd.concat([df, df_shifted.shift(i+1)], axis=1)
        df.dropna(axis=0, inplace=True)
    
    cols = list(df.columns)[1:]
    for i, col in enumerate(cols):
        cols[i] = 'shift_{}'.format(i+1)
    cols = [df.columns[0]] + cols
    df.columns = cols

    data = df.loc[:, ~df.columns.isin(['unshifted'])]
    X = np.array(data).reshape(data.shape[0], data.shape[1], 1)
    y = df.unshifted
    
    return X, y

def model(X, y):
    
    test_size = 60
    X_train, y_train = X[:-test_size], y[:-test_size]
    X_test, y_test = X[-test_size:], y[-test_size:]
    
    model = Sequential()
    model.add(LSTM(
        units={{choice([1,4,8,16,32,64,128,256,512])}},
        activation={{choice(['sigmoid', 'tanh', 'relu', 'linear'])}},
        use_bias={{choice([False, True])}},
        input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dense(1))
    
    optimizer = Adadelta(lr={{uniform(0,10)}})
    
    model.compile(loss='mean_squared_error',
                  optimizer=optimizer,
                  metrics=['mse'])

    model.fit(X_train, y_train,
              epochs=10,
              verbose=0,
              validation_data=(X_test, y_test))
    score, mse = model.evaluate(X_test, y_test, verbose=0)
    print('test mse: {}'.format(mse))
    return {'loss': mse, 'status': STATUS_OK, 'model': model}

X, y = data()

best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      algo=rand.suggest,
                                      max_evals=25,
                                      trials=Trials(),
                                      notebook_name='hyperopt_randsuggest')

print("Evalutation of best performing model:")
print(best_model.evaluate(X, y))
print(best_run)

Using TensorFlow backend.


>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from hyperopt import Trials, STATUS_OK, tpe, rand
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.layers import Dense, LSTM, Dropout, Activation
except:
    pass

try:
    from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from keras import backend as K
except:
    pass

try:
    import datetime as dt
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from sklearn.preprocessing import MinMaxScaler
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'units': hp.ch

In [ ]:
# Dropout 0:0, 1:0.01, 2:0.02, 3:0.05, 4:0.1
# activation 0:'sigmoid', 1:'tanh', 2:'relu', 3:'linear'
# units 0:1, 1:4, 2:8, 3:16, 4:32, 5:64, 6:128, 7:256, 8:512
# use_bias 0: False, 1: True

#optimizer = Adamax(lr={{uniform(0,1)}})
# defaults
# SGD lr 0.01 SKIP always high RMSE
# RMSprop lr 0.001
# Adam lr 0.001
# Adamax lr 0.002

# keras recommends leaving alone:
# Adagrad lr 0.001
# Adadelta lr 1.0
# Nadam lr 0.002